In [0]:
# Data preprocessing
cat_cols = ['main_category_en', 'main_category', 'categories_en',
            'categories', 'pnns_gp_2', 'pnns_gp_1']
sel_cat_cols = ['pnns_gp_1']
col_model = df[sel_cat_cols + main_nut_comp_cols + ['nutriscore']].columns

df_X = pd.get_dummies(df.loc[indTT,col_model])
X_cols = df_X.columns
print("---df_X nan values/shape:", df_X.isna().sum().sum(), df_X.shape)
X = df_X.fillna(0).values

df_y = serTT.map({'energy-kj_100g': 0,
                  'energy-kcal_100g': 1}).loc[indTT]
print("---df_y v_counts (0: kj, 1: kcal)\n", df_y.value_counts())
y = df_y.values

print("---X and y shapes:\n", X.shape, y.shape)

In [0]:
# Fitting grid search with cross-validation
Xtr, Xte, ytr, yte = train_test_split(X, y)
knn = KNeighborsClassifier()
knn_params = {'n_neighbors': [3,5,7]}
skf = StratifiedKFold(shuffle=True)
gs = GridSearchCV(knn, knn_params, cv=skf, scoring='accuracy')
gs.fit(Xtr, ytr)

# Printing the results of grid search
print('---'*20+'\n', gs.best_params_, '\n'+'---'*20)
res = gs.cv_results_ 
for mean, std, k in zip(res['mean_test_score'], res['std_test_score'], res['params']):
    print('for {} : mean accuracy : {:.5}, +/- {:.3}'.format(k, mean, std))
plt.gcf().set_size_inches(9,3)

''' oooooo knn 7 - X.fillna(0) oooooo
-- accuracy:  0.990491324673405
-- confusion matrix:  [267595    254   2319    427]'''
# Final prediction's performance of knn with best k
print('oooooo knn oooooo')
# ypr = gs.predict(Xte)
print("-- accuracy: ", metrics.accuracy_score(yte, ypr))
# TN (kj), FP, FN , TP (kcal)
print("-- confusion matrix: ", np.ravel(metrics.confusion_matrix(yte, ypr)))

# Comparison with naive baselines
for s in ['stratified', 'most_frequent', 'prior', 'uniform']:
    print('oooooo dummy ' + s + ' oooooo')
    dum = DummyClassifier(strategy=s).fit(Xtr, ytr)
    ypr_dum = dum.predict(Xte)
    print("-- accuracy: ", metrics.accuracy_score(yte, ypr_dum))
    print("-- confusion matrix: ",
          np.ravel(metrics.confusion_matrix(yte, ypr_dum)))